In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#there are two csv files. This one from anime reccomendation project from myanimelist.net API. https://www.kaggle.com/CooperUnion/anime-recommendations-database
#Looking for movies list, highlyrated shoes and the longest (and not bad, not kids) shows in the library to watch.

inpath='resources/anime.csv'
mal_csv=pd.read_csv(inpath)

# Content from the CSV CREATOR
#     anime_id - myanimelist.net's unique id identifying an anime.
#     name - full name of anime.
#     genre - comma separated list of genres for this anime.
#     type - movie, TV, OVA, etc.
#     episodes - how many episodes in this show. (1 if movie).
#     rating - average rating out of 10 for this anime.
#     members - number of community members that are in this anime's
#     "group".


In [3]:
#Cleanup the data and mapping the episodes[elements] from string to float

mal_data=mal_csv.loc[mal_csv['episodes'] != 'Unknown', ]
mal_data['episodes']=mal_data['episodes'].astype(float)
mal_data.head()

C:\Users\Bill\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1.0,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64.0,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51.0,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24.0,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51.0,9.16,151266


In [5]:
#Because my anime list had a long history for a website, and a vast library, some series information may not get updated as often. This dataframe is to look more closely at the unknown.
# These unknown may meant 'Ongoing' or sth else. At least two show on this list: One Piece and Conon had not ended yet. Both with over 900 episodes. 

unk_mal=mal_csv.loc[mal_csv['episodes'] == 'Unknown', ].loc[mal_csv['genre'] != 'Hentai', ]
unk_mal=unk_mal.sort_values(by=['members','rating','episodes'], ascending=False)

,anime_id,name,genre,type,episodes,rating,members
615,1735,Naruto: Shippuuden,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,Unknown,7.94,533578
74,21,One Piece,"Action, Adventure, Comedy, Drama, Fantasy, Sho...",TV,Unknown,8.58,504862
10899,25777,Shingeki no Kyojin Season 2,"Action, Drama, Fantasy, Shounen, Super Power",TV,Unknown,NaN,170054
252,235,Detective Conan,"Adventure, Comedy, Mystery, Police, Shounen",TV,Unknown,8.25,114702
1930,30694,Dragon Ball Super,"Action, Adventure, Comedy, Fantasy, Martial Ar...",TV,Unknown,7.40,111443
...,...,...,...,...,...,...,...
4975,33240,Chi&#039;s Sweet Home (2016),Slice of Life,TV,Unknown,6.57,3437
10974,32212,Ensemble Stars!,"Music, Shoujo",TV,Unknown,NaN,3317
10947,32086,Blame! Movie,"Action, Drama, Mecha, Psychological, Sci-Fi, S...",Movie,Unknown,NaN,3251
5920,31049,Beyblade Burst,"Action, Adventure, Sci-Fi, Shounen, Sports",TV,Unknown,6.31,3221


In [8]:
#Looking at how many movies are listed in the api.

mal_movie=mal_data.loc[mal_data['type'] == 'Movie', ]
mal_movie

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1.0,9.37,200630
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1.0,9.10,72534
11,28851,Koe no Katachi,"Drama, School, Shounen",Movie,1.0,9.05,102733
15,199,Sen to Chihiro no Kamikakushi,"Adventure, Drama, Supernatural",Movie,1.0,8.93,466254
18,12355,Ookami Kodomo no Ame to Yuki,"Fantasy, Slice of Life",Movie,1.0,8.84,226193
...,...,...,...,...,...,...,...
11738,3038,Senya Ichiya Monogatari,"Fantasy, Hentai",Movie,1.0,6.18,1394
11805,2755,Cleopatra,"Hentai, Historical",Movie,1.0,6.06,1709
12244,11141,Blue Seagull,"Action, Hentai",Movie,1.0,4.60,337
12258,20007,Hi Gekiga Ukiyoe Senya Ichiya,"Action, Hentai",Movie,1.0,1.92,129


In [9]:
#Looking at how many tv in the api that was loved aboved all with ratings over 9 out of ten. (after anime with unknown episodes are removed).

mal_tv=mal_data.loc[mal_data['type'] == 'TV', ]
mal_tv.loc[mal_tv['rating'] > 9, ]

,anime_id,name,genre,type,episodes,rating,members
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64.0,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51.0,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24.0,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51.0,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10.0,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148.0,9.13,425855
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13.0,9.11,81109
10,4181,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",TV,24.0,9.06,456749
12,918,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,201.0,9.04,336376


In [10]:
#Looking at how many tv in the api that was extremely liked (8/10) but not the highest. (after anime with unknown episodes are removed).

mal_tv.loc[mal_tv['rating']>8].loc[mal_tv['rating']<9]

,anime_id,name,genre,type,episodes,rating,members
13,2904,Code Geass: Hangyaku no Lelouch R2,"Action, Drama, Mecha, Military, Sci-Fi, Super ...",TV,25.0,8.98,572888
14,28891,Haikyuu!! Second Season,"Comedy, Drama, School, Shounen, Sports",TV,25.0,8.93,179342
16,23273,Shigatsu wa Kimi no Uso,"Drama, Music, Romance, School, Shounen",TV,22.0,8.92,416397
17,24701,Mushishi Zoku Shou 2nd Season,"Adventure, Fantasy, Historical, Mystery, Seine...",TV,10.0,8.88,75894
19,1575,Code Geass: Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,25.0,8.83,715151
...,...,...,...,...,...,...,...
10102,31547,Ryouma 30 Seconds,Historical,TV,5.0,8.80,54
10472,29485,Talpy,"Fantasy, Kids",TV,12.0,8.25,38
10718,22219,Warera Salaryman Tou,"Comedy, Slice of Life",TV,26.0,8.67,57
10734,19839,Wei Qi Shao Nian 2,"Game, Historical",TV,26.0,8.22,86


In [14]:
#Looking at how many tv in the api that was generally liked (7/10). (after anime with unknown episodes are removed).

mal_tv.loc[mal_tv['rating']>7].loc[mal_tv['rating']<8]

,anime_id,name,genre,type,episodes,rating,members
534,996,Bishoujo Senshi Sailor Moon: Sailor Stars,"Adventure, Comedy, Drama, Fantasy, Magic, Roma...",TV,34.0,7.99,52586
537,11239,Hidamari Sketch x Honeycomb,"Comedy, School, Slice of Life",TV,12.0,7.99,12590
538,967,Hokuto no Ken,"Action, Drama, Martial Arts, Sci-Fi, Shounen",TV,109.0,7.99,40271
539,1695,Les Misérables: Shoujo Cosette,"Drama, Historical, Shoujo, Slice of Life",TV,52.0,7.99,9605
540,339,Serial Experiments Lain,"Dementia, Drama, Mystery, Psychological, Sci-F...",TV,13.0,7.99,215829
...,...,...,...,...,...,...,...
10606,6917,Tsuideni Tonchinkan,Comedy,TV,43.0,7.13,189
10736,12039,What&#039;s Michael? (TV),Comedy,TV,45.0,7.13,208
10788,10554,Yakyuukyou no Uta,"Action, Drama, Sports",TV,25.0,7.15,292
10798,30116,Yancharu Moncha,Kids,TV,80.0,7.33,38


In [16]:
#the longest-running anime (after anime with unknown episodes are removed).
#Cleaning up other 
#Anime with 600 to over a 1000 episodes are mostly either shorts or made for kids.
#Anime with over 6.5 is considered not bad.
#Anime community with over 4000 is better validated 


longtv = mal_tv.loc[mal_tv['episodes'] > 100].loc[mal_tv['episodes'] < 600].loc[mal_tv['rating'] > 6.5].loc[mal_tv['members'] > 4000]
longtv = longtv.sort_values(by=['members','rating','episodes'], ascending=False)
longtv

,anime_id,name,genre,type,episodes,rating,members
841,20,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220.0,7.81,683297
582,269,Bleach,"Action, Comedy, Shounen, Super Power, Supernat...",TV,366.0,7.95,624055
288,6702,Fairy Tail,"Action, Adventure, Comedy, Fantasy, Magic, Sho...",TV,175.0,8.22,584590
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148.0,9.13,425855
206,813,Dragon Ball Z,"Action, Adventure, Comedy, Fantasy, Martial Ar...",TV,291.0,8.32,375662
12,918,Gintama,"Action, Comedy, Historical, Parody, Samurai, S...",TV,201.0,9.04,336376
304,1482,D.Gray-man,"Action, Adventure, Comedy, Shounen",TV,103.0,8.20,334399
346,223,Dragon Ball,"Adventure, Comedy, Fantasy, Martial Arts, Shou...",TV,153.0,8.16,316102
690,249,InuYasha,"Action, Adventure, Comedy, Demons, Fantasy, Ma...",TV,167.0,7.89,281632
175,1604,Katekyo Hitman Reborn!,"Action, Comedy, Shounen, Super Power",TV,203.0,8.37,258103


,anime_id,name,genre,type,episodes,rating,members
929,2471,Doraemon (1979),"Adventure, Comedy, Fantasy, Kids, Sci-Fi, Shounen",TV,1787.0,7.76,14233
5360,6277,Manga Nippon Mukashibanashi (1976),"Fantasy, Historical",TV,1471.0,6.48,406
6296,12393,Oyako Club,"Comedy, Slice of Life",TV,1818.0,6.18,160
6580,8213,Hoka Hoka Kazoku,Comedy,TV,1428.0,6.05,194
9242,32448,Kirin Ashita no Calendar,"Historical, Kids",TV,1306.0,6.43,59
9243,23349,Kirin Monoshiri Yakata,Kids,TV,1565.0,5.56,116
9613,22221,Monoshiri Daigaku: Ashita no Calendar,Historical,TV,1274.0,6.80,112
10165,10241,Sekai Monoshiri Ryoko,Comedy,TV,1006.0,5.92,153
